# N08. Algortimos Genéticos y _One-Max_

__Borja González Seoane, Computación Inteligente y Ética de la IA. Cursó2022-23__

## Preámbulo

En esta parte de la práctica se trabajar ́a con algoritmos genéticos y se aplicarán a un problema sencillo, el problema _one-max_. Este problema consiste en encontrar una cadena de bits de longitud L que contenga el mayor número de unos posible. Por ejemplo, para L = 10, la cadena `1111111111` sería la solución óptima, con un total de 10 unos; mientras que la cadena 0101010101 sería una solución con un total de 5 unos.

Instalación:

```sh
pip install numpy
```

Recordatorio: las librerías necesarias pueden ser instaladas desde el propio _notebook_ mediante un comando _magic_ de Jupyter. Descomentar la siguiente línea y ejecutar:

In [2]:
# %pip install numpy

In [3]:
from typing import Callable, Tuple

import numpy as np

import time

## Implementar las piezas necesarias para el algoritmo

In [4]:
individuo_test = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])
display(individuo_test)

array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])

In [5]:
# funcion one max hace la suma de los unos y divide por la longitud del individuo para obtener el ratio de unos en el array individuo
def fun_one_max(individuo: np.ndarray) -> float:
    """
    Función de evaluación en particular para el problema One-Max.

    :param individuo: El individuo que evaluar:
    :return: Ratio de adecuación al entorno en base 0 a 1.
    """
    return np.sum(individuo) / len(individuo) # Ratio de unos en el array


test = fun_one_max(individuo_test) 
display(test)
assert test == 0.6, f"Función mal implementada. Resultado diferente al esperado."

0.6

## Implementar el algoritmo genético a partir de las piezas anteriores

In [6]:
# algoritmo genetico recibe la funcion de evaluacion, el tamaño del cromosoma, el tamaño de la poblacion, el numero de generaciones y la probabilidad de mutacion
def algoritmo_genetico(
    fun_eval: Callable[[np.ndarray], float],
    tam_cromosoma: int,
    tam_poblacion: int,
    num_generaciones: int,
    prob_mutacion: float = 0.01,
) -> Tuple[np.ndarray, float]:
    """
    Implementación de un algoritmo genético para la resolución de problemas de optimización.

    :param fun_eval: Función de evaluación a optimizar.
    :param tam_cromosoma: Tamaño del cromosoma de los individuos.
    :param tam_poblacion: Tamaño de la población.
    :param num_generaciones: Número de generaciones a evolucionar. Criterio de parada.
    :param prob_mutacion: Probabilidad de mutación de un gen.
    :return: Tupla con el mejor individuo encontrado y su valor de la función de evaluación.
    """

    mejor_individuo = None
    mejor_valor = 0.0

    # Inicialización de la población
    poblacion = np.random.randint(0, 2, (tam_poblacion, tam_cromosoma)) # Matriz de 0s y 1s de tamaño tam_poblacion x tam_cromosoma

    for _ in range(int(num_generaciones*tam_poblacion/2)):
        # Evaluación de la población
        valores = np.array([fun_eval(individuo) for individuo in poblacion]) # Evaluación de cada individuo de la población

        # Selección de los dos mejores individuos
        indices = np.argsort(valores)[::-1] # Orden descendente
        padres = poblacion[indices[:2]] # Los dos primeros son los mejores padres

        # Cruce de los padres
        punto_cruce = np.random.randint(1, tam_cromosoma) # Punto de cruce aleatorio
        hijos = np.array(
            [np.concatenate([padres[0, :punto_cruce], padres[1, punto_cruce:]])] 
            + [np.concatenate([padres[1, :punto_cruce], padres[0, punto_cruce:]])]
        ) # Un hijo se obtiene con la primera mitad de un padre y la segunda de otro

        # Mutación de los hijos
        for hijo in hijos:
            for i in range(tam_cromosoma):
                if np.random.rand() < prob_mutacion: # Si el número aleatorio es menor que la probabilidad de mutación se cambia el gen
                    hijo[i] = 1 - hijo[i]

        # Reemplazo de los peores individuos
        peores = np.argsort(valores)[:2]
        poblacion[peores] = hijos

        # Actualización del mejor individuo
        mejor_valor_gen = np.max(valores)
        if mejor_valor_gen > mejor_valor:
            mejor_individuo = poblacion[np.argmax(valores)] # Mejor individuo de la generación
            mejor_valor = mejor_valor_gen
        
    return mejor_individuo, mejor_valor

## Pruebas con problema _one-max_

In [12]:
# se ejecuta el algoritmo genetico con la funcion one max
(individuo1, valor1) = algoritmo_genetico(fun_one_max, 10, 100, 100)
display(individuo1)
display(valor1)
print("END 1")

# intentamos obtener el mejor individuo con un cromosoma de 1000 alelos
(individuo2, valor2) = algoritmo_genetico(fun_one_max, 1000, 100, 100)
display(individuo2)
display(valor2)
print("END 2")

# intentamos obtener el mejor individuo de una poblacion de 1000 individuos
start_chrono1 = time.time()
(individuo3, valor3) = algoritmo_genetico(fun_one_max, 10, 1000, 100)
end_chrono1 = time.time()
display(individuo3)
display(valor3)
print("Response time: ", end_chrono1 - start_chrono1)
print("END 3")

# intentamos obtener el mejor individuo de una poblacion de 1000 individuos y 1000 generaciones
start_chrono2 = time.time()
(individuo4, valor4) = algoritmo_genetico(fun_one_max, 10, 1000, 1000)
end_chrono2 = time.time()
display(individuo4)
display(valor4)
print("Response time: ", end_chrono2 - start_chrono2)
print("END 4")


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

1.0

END 1


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,

0.884

...

1.0

END 3


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

1.0

END 4
